In [1]:
# Adrian Soto
# 2016-09-13
# Institute for Advanced Computational Science
# Stony Brook University
#
# Code to generate networks for
# IACS challenge at Tapia16

import numpy as np
from random import randint
import networkx as nx

In [46]:
def genStarNet(N):
# Generate star graph with N nodes.
# i==0 is the central node

    # Initialize adjacency matrix
    A=np.zeros((N,N))

                
    # %%%   Constructing the network   %%%
    for i in range(1,N):
        A[0][i]=1    

        # Symmetrize and return
    return A + np.transpose(A)


In [47]:
def genBrdcRcvNet(Nb, Nr):
#
# Generate broadcasters-receivers network
# with Nb broadcasters and Nr receiver nodes.
# 

    # Initialize adjacency matrix
    A=np.zeros((Nb+Nr,Nb+Nr))
    
    # %%%   Constructing the network   %%%
    for ib in range(0,Nb):
        for ir in range(Nb,Nb+Nr):
            A[ib][ir]=1    

        # Symmetrize and return
    
    return A + np.transpose(A)

In [95]:
def board2ind(i,j):  
    # CHESSBOARD k indices for A
    
    #
    #  /j  0  1  2  3  4  5  6  7 
    # i\
    # 0|   0  1  2  3  4  5  6  7
    # 1|   8  9 10 11 12 13 14 15
    # 2|  16 17 18 19 20 21 22 23
    # 3|  24 25 26 27 28 29 30 31
    # 4|  32 33 34 35 36 37 38 39
    # 5|  40 41 42 43 44 45 46 47
    # 6|  48 49 50 51 52 53 54 55
    # 7|  56 57 58 59 60 61 62 63
    # 
    
    return 8*i+j


def ind2board(k):
    # For a given index k in (0,64), give
    # the corresponding i,j indices on the
    # 8x8 lattice.
    j=k%8
    i=k//8
    return i,j



def genKnightNet():
#
# Generate network of knight-jump connected 
# chessboard cells.
#
# (1) Find allowed DOWNWARD moves (increasing i) 
#     on the 8x8 lattice and populate the corresponding 
#     entries of the adjacency matrix
# (2) Symmetrize A (i.e. account for UPWARD moves)
#

        
    N=64     # num of nodes in entire network
    # Initialize adjacency matrix
    A=np.zeros((N,N))

    

                
    # %%%   Constructing the network   %%%
    # Types of downward move: RRD, LLD, RDD, LDD
    # 
    # Loop across entire chess board, starting from upper 
    # left corner and to the right.
    for i in range(0,8):
        for j in range(0,8):
            k=board2ind(i,j)
            
            
            # RRD
            iaux=i+1
            jaux=j+2
            if (iaux<=7 and jaux<=7):
                kaux=board2ind(iaux,jaux)
                A[k][kaux]=1
            
            # LLD
            iaux=i+1
            jaux=j-2
            if (iaux<=7 and jaux>=0):
                kaux=board2ind(iaux,jaux)
                A[k][kaux]=1
            
            # RDD
            iaux=i+2
            jaux=j+1
            if (iaux<=7 and jaux<=7):
                kaux=board2ind(iaux,jaux)
                A[k][kaux]=1
            
            # LDD
            iaux=i+2
            jaux=j-1
            if (iaux<=7 and jaux>=0):
                kaux=board2ind(iaux,jaux)
                A[k][kaux]=1
            

        # 2. Symmetrize and return
    return A + np.transpose(A)



In [48]:
def join2Nets(A1,A2):
    # 
    # Join two networks defined by 
    # adjacency matrices A1 and A2
    #  ⎡A1 0⎤
    #  ⎣0 A2⎦
    #
    N1=np.shape(A1)[0]
    N2=np.shape(A2)[0]
    # Ensure adjacency matrices are diagonal
    if (N1 != np.shape(A1)[1]):
        print "join2Nets -- ERROR: A1 not diagonal"
        exit()
    if (N2 != np.shape(A2)[1]):
        print "join2Nets -- ERROR: A2 not diagonal"
        exit()
        
    
    # Construct block diagonal matrix
    
    # First construct the 01 and 10 rectangular
    # zero matrices
    Z01=np.zeros((N1,N2))
    Z10=np.zeros((N2,N1))
    
    # Combine
    Atop   =np.append(A1,Z01, axis=1)
    Abottom=np.append(Z10,A2, axis=1)
    A=np.append(Atop, Abottom, axis=0)
    
    return A

In [45]:
def genLayNet(M,L,E):
#
# Generate layered network
# Construct model single-product commercial distribution network.
# The network is unweighted and undirected, so we will 
# work on upper triangle and symmetrize in the end.
#
# (1) Edges between layers 1 and 2: manufacturer-warehouse
#     condition: each manufacturer only trades the product with one warehouse
# (2) Edges between layers 2 and 3: warehouse-retail
#     condition: each retailer only trades the product with one warehouse
# (3)
#
# (4)

# Variables:
#    Nmnf: num of manufacturers
#    Nwrh: num of warehouses
#    Nshp: num of shops
#    Ncst: num of costumers
#
#    M: number of layers
#    L: np.array of length M containing num of nodes on each layer
#    E: np.array of length M-1 containing num of edges between all layers
#
#

    

    if (M != L.size):
        print "ERROR: number of layers and size of array L don't match. EXIT"
        exit()

        
    N=sum(L)     # num of nodes in entire network

    
    # Initialize adjacency matrix
    A=np.zeros((N,N))

            
    
    # %%%   Constructing the network   %%%
    
    
    # Since all nodes are labeled by their entry in A,
    # get the Left and Right boundaries of each layer.
    Lnodes=np.concatenate( ( np.array([0]), np.cumsum( L[0:M-1]) ) )
    Rnodes=Lnodes + L - 1
    
    
    # Loop over layers. We generate the nodes downstream so,
    # iterate from first layer to second last one.
    for iL in range(0,M-1):
        
        if (E[iL] > L[iL]*L[iL+1]):
            print "WARNING!! Too many edges for this layer have been requested"
            print "          Number of nodes top layer       =", L[iL]
            print "          Number of nodes bottom layer    =", L[iL]
            print "          Number of possible edges        =", L[iL]*L[iL+1]
            print "          Number of requested edges       =", E[iL]
            print "          These two layers will have a max of ", L[iL]*L[iL+1], " edges"
            print " "
        
        # Find left and right edges of top and bottom layers
        iTL=Lnodes[iL]
        iTR=Rnodes[iL]
        iBL=Lnodes[iL+1]
        iBR=Rnodes[iL+1]
        
        
        # Loop inside top layer and assign edges downstream
        # onto the bottom layer following a random distribution.
        # STEPS:
        #    0. shuffle top and bottom layer index arrays
        #    1. assign one edge to each node sequentially, so that
        #       all nodes have at least one edge. Start from a random
        #       position on the layer.
        #    2. Shuffle T and B index arrays again
        #    3. Add aditional edges until run out. Draw random pairs
        #       of integers from top and bottom layers, each pair
        #       forming an edge.
        #    4. Symmetrize A

        # 0. 
        
        EdgesRemaining=E[iL]
        Tnodes = np.arange(iTL,iTR+1)
        np.random.shuffle(Tnodes)
        Bnodes = np.arange(iBL,iBR+1)
        np.random.shuffle(Bnodes)
 
        # 1. Wire all nodes with at least one edge
        if (E[iL] < max(L[iL], L[iL+1])):
            print "ERROR -- not enough edges to make network fully connected. EXITING..."
            exit()
        else:
            for i in range(0,max(L[iL], L[iL+1])):
                # If index gets larger than layer size, 
                # loop around and get back to beginning.
                if (i//L[iL]>0):
                    iT=Tnodes[i-(i//L[iL])*L[iL]]
                else:
                    iT=Tnodes[i]
                
                if (i//L[iL+1]>0):
                    iB=Bnodes[i-(i//L[iL+1])*L[iL+1]]
                else:
                    iB=Bnodes[i]
                
                # Add edge
                A[iT][iB]=1
        
                EdgesRemaining -= 1 # max(L[iL], L[iL-1])
            

        
        
        # 2. Shuffle again
        np.random.shuffle(Tnodes)
        np.random.shuffle(Bnodes)

                
        # 3. Include remaining edges
        
        # Find node indices
        if (iTL==iTR):
            Tind=np.array([iTL])
        else:
            Tind=np.random.randint(iTL,iTR, EdgesRemaining)
        
        if (iBL==iBR):
            Bind=np.array([iBL])
        else:
            Bind=np.random.randint(iBL,iBR, EdgesRemaining)
        
        # Add edges
        for i in range(0,EdgesRemaining):
            A[Tind[i]][Bind[i]]=1
        

        # 4. Symmetrize and return
    return A + np.transpose(A)






In [ ]:
# Create multilayer network
L=np.array([10, 3, 10, 41])
E=np.array([10, 20, 100])
A=genLayNet(4,L,E)
np.savetxt('4layer_network.dat', A, fmt='%.1i')

In [96]:
# Create knight jump network
A=genKnightNet()
np.savetxt('knightjump_network.dat', A, fmt='%.1i')


In [132]:
# Create star network (better known as star graph)
A=genStarNet(64)
np.savetxt('star64_network.dat', A, fmt='%.1i')

In [49]:
# Create a broadcasters-receivers network
# with Nb broadcasters and Nr receivers.
Nb=4
Nr=60
A=genBrdcRcvNet(Nb, Nr)
np.savetxt('4broadcasters_network.dat', A, fmt='%.1i')

In [58]:
# Create multi-gang network based on 
# 2 Barabasi-Albert networks, often used
# for simulating social networks. 

# In this example we have 3-gangs. We construct
# 3 BA networks and then we join them. 
#
# Each BA network can be modeled with 2 parameters:
#    N: Number of nodes
#    C: 

# Parameter choice. Ensure N1+N2+N3=64
N1=20
C1=7
N2=16
C2=C1
N3=64-N1-N2
C3=C1

Gg1=nx.barabasi_albert_graph(N1,C1)
Ag1=nx.to_numpy_matrix(Gg1)

Gg2=nx.barabasi_albert_graph(N2,C2)
Ag2=nx.to_numpy_matrix(Gg2)

Ag12=join2Nets(Ag1, Ag2)

Gg3=nx.barabasi_albert_graph(N3,C3)
Ag3=nx.to_numpy_matrix(Gg3)

Ag=join2Nets(Ag12, Ag3)

Gg=nx.from_numpy_matrix(A)

np.savetxt('3gangs_network.dat', Ag, fmt='%.1i')
